In [1]:
type Nm = String

type State = Int
type Sigma = Char
data Gamma = N Nm     -- Non-terminal symbol
           | T Sigma  -- Terminal symbol
           | Z        -- initial stack symbol
           deriving (Eq, Show)
type Delta = [(State, Maybe Sigma, Maybe Gamma, [Gamma], State)]

-- (max state, transitions, initial state, final states)
type PDA = (State, Delta, State, [State])

In [14]:
delta :: PDA -> State -> Maybe Sigma -> Maybe Gamma -> [(State, [Gamma])]
delta (_, d, _, _) p ms mg =
  [(q,gs) | (p',ms',mg',gs,q)<-d, p==p', ms==ms', mg==mg']

In [15]:
step pda (p,   [],  [])   =
  [(q,   [], gs')        | (q,gs') <- delta pda p Nothing  Nothing ]
step pda (p, c:cs,   [])  =
  [(q, c:cs, gs')        | (q,gs') <- delta pda p Nothing  Nothing ] ++
  [(q,   cs, gs')        | (q,gs') <- delta pda p (Just c) Nothing ]
step pda (p,[],    g:gs) =
  [(q,   [], gs'++ g:gs) | (q,gs') <- delta pda p Nothing  Nothing ] ++
  [(q,   [], gs'++   gs) | (q,gs') <- delta pda p Nothing  (Just g)]
step pda (p, c:cs, g:gs) =
  [(q, c:cs, gs'++ g:gs) | (q,gs') <- delta pda p Nothing  Nothing ] ++
  [(q,   cs, gs'++ g:gs) | (q,gs') <- delta pda p (Just c) Nothing ] ++
  [(q, c:cs, gs'++   gs) | (q,gs') <- delta pda p Nothing  (Just g)] ++
  [(q,   cs, gs'++   gs) | (q,gs') <- delta pda p (Just c) (Just g)]

In [16]:
(p,cs,gs) |- (q,cs',gs') = \pda -> (q,cs',gs') `elem` step pda (p,cs,gs)

:type (|-)

(|-) :: (State, [Sigma], [Gamma]) -> (State, [Sigma], [Gamma]) -> PDA -> Bool

In [38]:
(|-*) (p,cs,gs) (q,cs',gs') (_,   n) | n <= 0 = (p,cs,gs)==(q,cs',gs')
(|-*) (p,cs,gs) (q,cs',gs') (_,   _) | (p,cs,gs)==(q,cs',gs') = True
(|-*) (p,cs,gs) (q,cs',gs') (pda, n) =
  or [(p1,cs1,gs1) |-* (q,cs',gs') $ (pda, n-1) | (p1,cs1,gs1) <- step pda (p,cs,gs)]

In [39]:
anbnPDA :: PDA
anbnPDA = (2,d,0,[2])
  where
    d = [ (0, e,__Z, [_A, Z],    0)
        , (0, e,__A, [_a,_A,_b], 0)
        , (0, a,__a, [],         0)
        , (0, e,__A, [],         1)
        , (1, b,__b, [],         1)
        , (1, e,__Z, [],         2)
        ]
    e = Nothing
    a = Just 'a'
    b = Just 'b'
    __Z = Just Z
    __A = Just _A
    __a = Just _a
    __b = Just _b

_A = N "A"
_a  = T 'a'
_b  = T 'b'

In [45]:
(0,"aabb",            [ Z]) |- (0,"aabb",         [_A, Z]) $ anbnPDA
(0,"aabb",         [_A, Z]) |- (0,"aabb",   [_a,_A,_b, Z]) $ anbnPDA
(0,"aabb",   [_a,_A,_b, Z]) |- (0, "abb",      [_A,_b, Z]) $ anbnPDA
(0, "abb",      [_A,_b, Z]) |- (0, "abb",[_a,_A,_b,_b, Z]) $ anbnPDA
(0, "abb",[_a,_A,_b,_b, Z]) |- (0,  "bb",   [_A,_b,_b, Z]) $ anbnPDA
(0,  "bb",   [_A,_b,_b, Z]) |- (1,  "bb",      [_b,_b, Z]) $ anbnPDA
(1,  "bb",      [_b,_b, Z]) |- (1,   "b",         [_b, Z]) $ anbnPDA
(1,   "b",         [_b, Z]) |- (1,    "",            [ Z]) $ anbnPDA
(1,    "",            [ Z]) |- (2,    "",              []) $ anbnPDA

True

True

True

True

True

True

True

True

True

In [44]:
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 0)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 1)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 2)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 3)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 4)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 5)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 6)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 7)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 8)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA, 9)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA,10)
(0,"aabb",[Z]) |-* (2,"",[]) $ (anbnPDA,11)

False

False

False

False

False

...

False

False

True

True

True

In [46]:
accepts cs pda@(_,_,q0,fs) n = or [(q0,cs,[Z]) |-* (f,"",[]) $ (pda, n) | f<-fs]

In [47]:
accepts "aabb" anbnPDA 10

True